<a target="_blank" href="https://colab.research.google.com/github/Sebelino/DD2424-project/blob/main/e1_sebastian.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%load_ext autoreload
%autoreload 2

import os
import socket

repo_name = "DD2424-project"
dep_filename = "download_dataset.py"
cwd = os.getcwd()
hostname = socket.gethostname()

print(f"Host: {hostname}")
print(f"You are here: {cwd}")

if os.path.exists(dep_filename):
    print(f"{dep_filename} is in your current directory. Assuming you are in the root of the repository.")
elif not os.path.exists(repo_name):
    print(f"Cloning repo and fetching {dep_filename}...")
    !git clone https://github.com/Sebelino/DD2424-project
    !cp DD2424-project/download_dataset.py .
else:
    !git -C DD2424-project pull --rebase
    !cp DD2424-project/download_dataset.py .

Host: deeplearning-vm
You are here: /home/silpasoninallacheruvu/DD2424-project
download_dataset.py is in your current directory. Assuming you are in the root of the repository.


# Train model

In [ ]:
# from dataclasses import asdict
# from util import json_dumps
# from evaluation import evaluate_test_accuracy_and_misclassified, \
#     evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
# from run import run
# %reload_ext autoreload
# %autoreload 2

# from determinism import Determinism

# determinism = Determinism(seed=42).sow()  # Keep this before any torch import

# from training import TrainParams, NagParams, AdamParams

# baseline_params = TrainParams(
#     seed=determinism.seed,
#     batch_size=32,
#     architecture="resnet50",
#     n_epochs=10,
#     optimizer=NagParams(
#         learning_rate=0.01,
#         weight_decay=1e-4,
#         momentum=0.9,
#     ),
#     freeze_layers=True,
#     unfreezing_epochs=None,
#     validation_freq=1,
#     unfreeze_last_l_blocks = 1,
#     time_limit_seconds=None,
#     val_acc_target=None,
# )

# invalidate_cache_entry = True
# if invalidate_cache_entry:
#     run.call_and_shelve(baseline_params).clear()

# # First plot: Training and validation accuracy
# for l in range(1,6):
#     baseline_params.unfreeze_last_l_blocks = l
#     print("Training with params:")
#     print(baseline_params.pprint())
#     result = run(baseline_params)

#     print("Training result:")
#     print(result.pprint())
#     evaluate_with_train_val_plot(result)

#     # Second plot: Rerun training multiple times with different seed
#     run_with_different_seeds(baseline_params, trials=3)

#     # Third plot: Compare NAG with Adam
#     experimental_params = baseline_params.copy()
#     experimental_params.optimizer = AdamParams(learning_rate=0.001)
    
#     param_sets = {
#         "Baseline (NAG)": baseline_params,
#         "Experiment (Adam)": experimental_params,
#     }
    
#     run_comparison(param_sets, trials=3)


# Train semi-supervised model

In [ ]:
%reload_ext autoreload
%autoreload 2

from training import TrainParams, NagParams
from evaluation import evaluate_test_accuracy_and_misclassified, \
    evaluate_with_train_val_plot, run_with_different_seeds, run_comparison
from run import run
from caching import invalidate_cache_entry
from datasets import DatasetParams
from augmentation import AugmentationParams

from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

run_args = (dataset_params, baseline_params)

# Set invalidate=True to invalidate a single cache entry
invalidate_cache_entry(run, run_args, invalidate=False)

# First plot: Training and validation accuracy
print("Training with params:")
print(baseline_params.pprint())
result = run(*run_args)

print("Training result:")
print(result.pprint())
evaluate_with_train_val_plot(result)

In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

run_dataset_comparison(param_sets, baseline_params)


In [ ]:
from evaluation import run_dataset_comparison

def tweak(params: DatasetParams, labelled_data_fraction):
    params = params.copy()
    params.labelled_data_fraction = labelled_data_fraction
    return params
 
#plot: Compare unsupervised learning for different fractions of unlabelled data along with threshold
labelled_data_fractions = [ 0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
param_sets = {f"labelled_data_fraction={data_fraction}": tweak(dataset_params, labelled_data_fraction=data_fraction) for data_fraction in labelled_data_fractions}

baseline_params.pseudo_threshold = None
run_dataset_comparison(param_sets, baseline_params)


In [12]:
from evaluation import evaluate_test_accuracy
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

determinism = Determinism(seed=42).sow()  # Keep this before any torch import

dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=0.5, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 1,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

#plot: Compare unsupervised learning for different values of unsup_weight
unsup_weights = [ 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for unsup_weight in unsup_weights:
    baseline_params.unsup_weight = unsup_weight
    training_params = deepcopy(baseline_params)

    print("\n=====================================")
    print(f"Training with unsup_weight={unsup_weight}")
    print("=====================================")
    # print(training_params.pprint())

    trainer = try_loading_trainer(dataset_params, training_params, determinism)

    test_dataset = load_dataset("test", Trainer.make_base_transform(training_params))
    test_loader = DataLoader(
        test_dataset,
        batch_size=dataset_params.batch_size,
        shuffle=False,
        num_workers=3,
        persistent_workers=False,
        pin_memory=True,
        worker_init_fn=Determinism.data_loader_worker_init_fn(dataset_params.shuffler_seed),
    )

    print(f"Test size: {len(test_loader.dataset)}")
    final_test_acc = evaluate_test_accuracy(trainer, test_loader)
    print(f"Final Test Accuracy: {final_test_acc:.2f}%")



Training with unsup_weight=0.1
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.2638, Train Acc: 32.47%, Val Acc: 80.65%
Epoch [2/5], Loss: 2.0929, Train Acc: 77.90%, Val Acc: 89.10%
Epoch [3/5], Loss: 0.9740, Train Acc: 86.71%, Val Acc: 91.42%
Epoch [4/5], Loss: 0.5534, Train Acc: 90.58%, Val Acc: 91.14%
Epoch [5/5], Loss: 0.4033, Train Acc: 93.83%, Val Acc: 92.10%
Total elapsed: 99.82s, average per update step: 0.09s
Test size: 3669


Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Final Test Accuracy: 89.94%

Training with unsup_weight=0.2
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


...

Final Test Accuracy: 89.97%

Training with unsup_weight=1.0
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/465 [00:00<?, ?it/s]

Epoch [1/5], Loss: 3.2638, Train Acc: 32.47%, Val Acc: 80.65%
Epoch [2/5], Loss: 2.0929, Train Acc: 77.90%, Val Acc: 89.10%
Epoch [3/5], Loss: 0.9740, Train Acc: 86.71%, Val Acc: 91.42%
Epoch [4/5], Loss: 0.5534, Train Acc: 90.58%, Val Acc: 91.14%
Epoch [5/5], Loss: 0.5366, Train Acc: 93.76%, Val Acc: 91.96%
Total elapsed: 98.99s, average per update step: 0.09s
Test size: 3669


Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Final Test Accuracy: 89.86%


# Main Testing loop for semi-supervised model

In [4]:
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

%reload_ext autoreload
%autoreload 2


determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=1.0, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=5,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 0.5,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

fractions = [1.0] #[0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
epochs_list = [10, 20, 30]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for num_epochs in epochs_list:
        training_params = deepcopy(baseline_params)
        training_params.n_epochs = num_epochs

        print("\n=====================================")
        print(f"Training with fraction={fraction}, epochs={num_epochs}")
        print("=====================================")
        # print(training_params.pprint())
        # print(dataset_params.pprint())

        trials = 3
        # run_with_different_seeds(dataset_params, training_params, determinism, trials)
        evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )


Training with fraction=1.0, epochs=10
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/1150 [00:00<?, ?it/s]

Epoch [1/10], Loss: 2.4100, Train Acc: 57.17%
Epoch [2/10], Loss: 0.6139, Train Acc: 88.23%
Epoch [3/10], Loss: 0.3078, Train Acc: 93.10%
Epoch [4/10], Loss: 0.2065, Train Acc: 95.60%
Epoch [5/10], Loss: 0.1458, Train Acc: 96.17%
Epoch [6/10], Loss: 0.1062, Train Acc: 97.74%
Epoch [7/10], Loss: 0.0831, Train Acc: 98.23%
Epoch [8/10], Loss: 0.0655, Train Acc: 98.70%
Epoch [9/10], Loss: 0.0567, Train Acc: 98.80%
Epoch [10/10], Loss: 0.0439, Train Acc: 99.27%
Total elapsed: 134.93s, average per update step: 0.10s


...

Test Accuracy: 92.723 %
Trying to load trainer from disk...
Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/3450 [00:00<?, ?it/s]

Epoch [1/30], Loss: 2.3934, Train Acc: 58.75%
Epoch [2/30], Loss: 0.6276, Train Acc: 87.85%
Epoch [3/30], Loss: 0.3104, Train Acc: 93.12%
Epoch [4/30], Loss: 0.2155, Train Acc: 94.89%
Epoch [5/30], Loss: 0.1446, Train Acc: 96.77%
Epoch [6/30], Loss: 0.1047, Train Acc: 97.83%
Epoch [7/30], Loss: 0.0821, Train Acc: 98.64%
Epoch [8/30], Loss: 0.0665, Train Acc: 98.75%
Epoch [9/30], Loss: 0.0531, Train Acc: 98.91%
Epoch [10/30], Loss: 0.0452, Train Acc: 99.29%
Epoch [11/30], Loss: 0.0344, Train Acc: 99.59%
Epoch [12/30], Loss: 0.0294, Train Acc: 99.73%
Epoch [13/30], Loss: 0.0282, Train Acc: 99.62%
Epoch [14/30], Loss: 0.0204, Train Acc: 99.73%
Epoch [15/30], Loss: 0.0194, Train Acc: 99.78%
Epoch [16/30], Loss: 0.0172, Train Acc: 99.84%
Epoch [17/30], Loss: 0.0214, Train Acc: 99.65%
Epoch [18/30], Loss: 0.0146, Train Acc: 99.92%
Epoch [19/30], Loss: 0.0129, Train Acc: 99.97%
Epoch [20/30], Loss: 0.0119, Train Acc: 99.86%
Epoch [21/30], Loss: 0.0103, Train Acc: 99.89%
Epoch [22/30], Loss: 0

Evaluating:   0%|          | 0/115 [00:00<?, ?it/s]

Test Accuracy: 92.968 %
Test Accuracy Mean: 92.84 %
Test Accuracy Standard Error: 0.07 percentage points


In [9]:
from run import try_loading_trainer
from augmentation import AugmentationParams
from evaluation import evaluate_final_test_accuracy, run_with_different_seeds
from torch.utils.data import DataLoader
from datasets import load_dataset, DatasetParams
from training import Trainer, TrainParams, NagParams
from copy import deepcopy
from caching import invalidate_cache_entry
from determinism import Determinism

%reload_ext autoreload
%autoreload 2


determinism = Determinism(seed=42).sow()  # Keep this before any torch import
   
dataset_params = DatasetParams(
    splitting_seed=determinism.seed,
    shuffler_seed=determinism.seed,
    batch_size=32,
    #trainval_size=None,  # Load all training data
    class_fractions=(1.0,) * 37,
    #trainval_size=400,  # Load a subset
    validation_set_fraction=0.2,  # 20 % of trainval set
    labelled_data_fraction=1.0, # full labelled data first
)

baseline_params = TrainParams(
    seed=determinism.seed,
    architecture="resnet50",
    n_epochs=20,
    optimizer=NagParams(
        learning_rate=0.004713842776302303,
        weight_decay=4.287389519996026e-06,
        momentum=0.7222321829989696,
    ),
    freeze_layers=True,
    unfreezing_epochs=(2, 4),
    validation_freq=1,
    time_limit_seconds=None,
    val_acc_target=None,
    unfreeze_last_l_blocks = 2,
    pseudo_threshold = 0.95,
    unsup_weight = 0.5,
    augmentation = AugmentationParams(
        enabled=False,
        transform=None
    ),
    fixmatch=True,
    use_scheduler=True,
    scheduler_type="plateau"
)

fractions = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
pseudo_threshold_list = [None, 0.5, 0.7]

for fraction in fractions:
    dataset_params.labelled_data_fraction = fraction
    for pseudo_threshold in pseudo_threshold_list:
        training_params = deepcopy(baseline_params)
        training_params.pseudo_threshold = pseudo_threshold

        print("\n=====================================")
        print(f"Training with fraction={fraction}, pseudo_threshold={pseudo_threshold}")
        print("=====================================")
        # print(training_params.pprint())
        # print(dataset_params.pprint())


        trials = 3
        evaluate_final_test_accuracy(
            dataset_params,
            training_params,
            determinism,
            trials,
            display_misclassified=False,
        )


Training with fraction=0.1, pseudo_threshold=None
Test size: 3669
Trying to load trainer from disk...


/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/2320 [00:00<?, ?it/s]

Epoch [1/20], Loss: 7.2307, Train Acc: 2.70%
Epoch [2/20], Loss: 4.2892, Train Acc: 2.70%
Epoch [3/20], Loss: 3.6401, Train Acc: 2.70%
Epoch [4/20], Loss: 3.3148, Train Acc: 2.97%
Epoch [5/20], Loss: 3.0609, Train Acc: 8.11%
Epoch [6/20], Loss: 2.7396, Train Acc: 22.70%
Epoch [7/20], Loss: 2.4075, Train Acc: 40.81%
Epoch [8/20], Loss: 1.9713, Train Acc: 61.35%
Epoch [9/20], Loss: 1.6732, Train Acc: 67.84%
Epoch [10/20], Loss: 1.3504, Train Acc: 76.76%
Epoch [11/20], Loss: 1.0901, Train Acc: 83.78%
Epoch [12/20], Loss: 0.9588, Train Acc: 88.92%
Epoch [13/20], Loss: 0.7693, Train Acc: 91.35%
Epoch [14/20], Loss: 0.6729, Train Acc: 89.46%
Epoch [15/20], Loss: 0.5608, Train Acc: 93.51%
Epoch [16/20], Loss: 0.4847, Train Acc: 96.22%
Epoch [17/20], Loss: 0.5023, Train Acc: 95.41%
Epoch [18/20], Loss: 0.3464, Train Acc: 96.76%
Epoch [19/20], Loss: 0.4918, Train Acc: 97.03%
Epoch [20/20], Loss: 0.4935, Train Acc: 97.84%
Total elapsed: 601.15s, average per update step: 0.11s


...

/opt/python/3.10/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trainer not found. Retraining...
[Trainer] Unfroze last 2 blocks


Update step:   0%|          | 0/2320 [00:00<?, ?it/s]

Epoch [1/20], Loss: 3.3718, Train Acc: 24.16%
Epoch [2/20], Loss: 2.5946, Train Acc: 71.76%
Epoch [3/20], Loss: 1.5495, Train Acc: 83.17%
Epoch [4/20], Loss: 1.1517, Train Acc: 88.69%
Epoch [5/20], Loss: 1.8256, Train Acc: 82.99%
Epoch [6/20], Loss: 1.5358, Train Acc: 83.62%
Epoch [7/20], Loss: 1.7444, Train Acc: 86.43%
Epoch [8/20], Loss: 2.0613, Train Acc: 86.88%
Epoch [9/20], Loss: 1.8326, Train Acc: 91.13%
Epoch [10/20], Loss: 1.6589, Train Acc: 92.76%
Epoch [11/20], Loss: 1.5641, Train Acc: 92.04%
Epoch [12/20], Loss: 1.5056, Train Acc: 96.02%


KeyboardInterrupt: 

In [ ]:
from evaluation import show_misclassified

final_test_acc, misclassified_samples = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples)}")
show_misclassified(misclassified_samples)

In [ ]:
from evaluation import show_misclassified

final_test_acc_tau, misclassified_samples_tau = evaluate_test_accuracy_and_misclassified(trainer, test_loader, test_dataset)
print(f"Final Test Accuracy: {final_test_acc_tau:.2f}%")
print(f"Number of misclassified samples: {len(misclassified_samples_tau)}")
show_misclassified(misclassified_samples_tau)

In [ ]:
misclassified_diff = list(set(misclassified_samples).symmetric_difference(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff)}")
show_misclassified(misclassified_diff)

In [ ]:
misclassified_diff2 = list(set(misclassified_samples) - set(misclassified_samples_tau))
print(f"Difference of misclassified samples: {len(misclassified_diff2)}")
show_misclassified(misclassified_diff2)